In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import pickle as pkl

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [2]:
with open('datasets/singletons_af_w_pza_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)
    
print(len(graph_dict['train']) + len(graph_dict['test']))

664


### Parameters

Best param value ranges from WandB sweeps

- Cutoff distance = 11 - 12.5
- Dropout = 0.4-0.5
- Edge weights = "exp"
- Edge weight lambda = 2
- Hidden channels = 320
- Learning rate = 3.5e-5 - 4.5e-5
- Weight decay = 1e-6 - 1e-5

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)


# logging params (only used for wandb metrics)
n_samples = len(graph_dict['train']) + len(graph_dict['test'])
cutoff_distance = 12

# gcn params - from best wandb sweep
num_node_features = 16
batch_size = 256
hidden_channels = 320
dropout = 0.5

edge_weight_func = "exp"
edge_weight_lambda = 2

learning_rate = 4e-5
wd = 5e-5
epochs = 942


wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [4]:
project = 'train-final-model'

In [5]:
run_name = f'Run 6 - {epochs} epochs'

model = run_model.pnca_GCN_vary_graph(
            self_loops = False,
            cutoff_distance = cutoff_distance,
            edge_weight_func = edge_weight_func,
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = hidden_channels,
            learning_rate = learning_rate,
            wd = wd,
            dropout = dropout,
            lr_scheduling=False,
            epochs = epochs,
            graph_dict= graph_dict,
            normalise_ews=True,
            lambda_param= edge_weight_lambda,
            early_stop=False,
            wandb_params={
              'use_wandb': True, 
              'wandb_project': f'{project}', 
              'wandb_name': f'{run_name}',
              'n_samples': n_samples,
              'sweep': False
              }
        )

Adjusting edge index and attaching edge weights for cutoff distance 12
Using CUDA


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: dylan-home. Use `wandb login --relogin` to force relogin


Epoch: 010, Train Acc: 0.5345, Test Acc: 0.4800, Train Loss: 0.6919, Test Loss: 0.6923
Epoch: 020, Train Acc: 0.5388, Test Acc: 0.5050, Train Loss: 0.6894, Test Loss: 0.6907
Epoch: 030, Train Acc: 0.5280, Test Acc: 0.5100, Train Loss: 0.6884, Test Loss: 0.6905
Epoch: 040, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6868, Test Loss: 0.6902
Epoch: 050, Train Acc: 0.5302, Test Acc: 0.5150, Train Loss: 0.6873, Test Loss: 0.6896
Epoch: 060, Train Acc: 0.5345, Test Acc: 0.5150, Train Loss: 0.6857, Test Loss: 0.6883
Epoch: 070, Train Acc: 0.5409, Test Acc: 0.5200, Train Loss: 0.6845, Test Loss: 0.6874
Epoch: 080, Train Acc: 0.5323, Test Acc: 0.5150, Train Loss: 0.6831, Test Loss: 0.6868
Epoch: 090, Train Acc: 0.5582, Test Acc: 0.5100, Train Loss: 0.6817, Test Loss: 0.6852
Epoch: 100, Train Acc: 0.5323, Test Acc: 0.5150, Train Loss: 0.6804, Test Loss: 0.6843
Epoch: 110, Train Acc: 0.5711, Test Acc: 0.5100, Train Loss: 0.6774, Test Loss: 0.6819
Epoch: 120, Train Acc: 0.5539, Test Acc: 0.

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Test Accuracy,▁▁▁▁▁▁▃▃▅▆▆▆▆▆▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██▇▇█████▇
Test F1,▃▂▃▃▃▃▃▃▄▂▃▂▂▃▁▄▄▅▅▃▆▇▇▆▇▇▆▆▇▇█▇▆▆███▇██
Test Loss,██████▇▇▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▂▁▁▂
Test Sensitivity,█▇████▆▆▄▂▃▂▂▃▁▃▃▃▄▂▄▅▅▄▅▅▄▄▄▅▅▄▃▄▅▅▅▄▅▅
Test Specificity,▁▂▁▁▁▁▃▃▆█▇██▇█▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇███▇▇▇█▇▇
Train Accuracy,▁▂▁▂▁▂▃▄▆▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇███▇██▇███████
Train F1,▂▁▂▂▂▂▃▄▅▂▅▄▄▅▂▅▅▆▆▅▇▇▇▆▇▇▇▇▇▇█▇▇▇▇██▇██
Train Loss,██████▇▇▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
Train Sensitivity,█▆████▆▆▄▂▃▃▃▄▁▃▃▄▄▃▅▅▅▄▅▅▄▄▄▅▅▄▄▄▄▅▅▄▅▅
Train Specificity,▁▃▁▂▁▂▄▄▇▇▇██▇█▇█▇▇█▇▇▇█▇▇███▇██████▇███
Test Accuracy,0.795


In [6]:
torch.save(model, f'saved_models/{project}/{run_name}')